In [1]:
import numpy as np
import scipy as sp
from scipy.special import loggamma as lg
from scipy.integrate import quad, quad_vec, fixed_quad
#from quadpy import quad

In [11]:
def log_bessel(v, z):
    
    z_tmp = z
    
    lead = 0.5*np.log(np.pi)-lg(v+0.5)-v*np.log(2*z)-z
    
    beta = lambda v : 2*(v+0.5)/(2*v+1)

    first = lambda x, v, z : beta(v) * np.exp( -x**beta(v) ) * ( 2 * z + x**beta(v) )**(v-0.5) * x**(v-0.5)
    second = lambda x, v, z : np.exp(-1/x) * x**(-2*v-1) * ( 2 * z * x + 1 )**(v-0.5)

    def integral(x, v):
        
        return first(x,v,z_tmp)+second(x,v,z_tmp)
    
    Q = np.vectorize(quad)(integral, 0, 1, args=(v))
    
    return lead+np.log(Q[0])

In [33]:
v = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
vals=np.exp(log_bessel(v, 0.1))

In [34]:
error = np.zeros(len(v)-2)
for i in range(2, len(v)):
    val_rec = vals[i-2] + 2*(v-1) / 10
    error[i-2] = np.abs(vals[i] / val_rec[i])
    
error

array([   779.22112156,   2396.7553599 ,   4802.51668147,   8002.64928728,
        12002.49931238,  16802.39981893,  22402.33322217,  28802.28563777,
        36002.2499442 ,  44002.22217979,  52802.19996667,  62402.18179132,
        72802.16664457,  84002.15382766,  96002.14284144, 108802.13331984,
       122402.12498828, 136802.11763679])